Using the titanic data, in your classification-exercises repository, create a notebook, decision_tree.ipynb where you will do the following:


In [ ]:
import pandas as pd
import env
import acquire
import prepare


1. What is your baseline prediction? What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.



2. Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)



3. Evaluate your in-sample results using the model score, confusion matrix, and classification report.



4. Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.


5. Run through steps 2-4 using a different max_depth value.



6. Which model performs better on your in-sample data?



7. Which model performs best on your out-of-sample data, the validate set?



#### Bonus
1. Work through these same exercises using the Telco dataset.

2. Experiment with this model on other datasets with a higher number of output classes.